# 人口データの取得・可視化・分析


In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import requests
# データ処理に用いるライブラリ
import numpy as np
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
)

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

In [ ]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch07").resolve()

## 人口と国勢調査

- [国勢調査](https://www.e-stat.go.jp/statistics/00200521)
- [令和2 年国勢調査（2020 年）の公表](https://www.stat.go.jp/data/kokusei/2020/pdf/schedule.pdf)


In [ ]:
pop_statsDataId = "0003410380"
pop_meta = get_metainfo(appId, pop_statsDataId)
pop_metadata = pop_meta["GET_META_INFO"]["METADATA_INF"]
pop_total_num = pop_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
pop_total_num

In [ ]:
pop_data = get_statsdata(appId, pop_statsDataId)
pop_value = colname_to_japanese(cleansing_statsdata(pop_data))
pop_value.columns

### 人口ピラミッド


In [ ]:
pop_value["男女_時系列"].unique()

In [ ]:
pyramid_cols = ["年齢（５歳階級）_時系列コード", "年齢（５歳階級）_時系列", "男女_時系列", "値"]
pyramid_cond = pop_value["表章項目"] == "人口"
pyramid_cond &= pop_value["男女_時系列"] != "総数"
pyramid_cond &= pop_value["年齢（５歳階級）_時系列"] != "総数"
pyramid_cond &= ~ pop_value["年齢（５歳階級）_時系列"].str.contains("再掲")
pyramid_cond &= pop_value["時間軸（調査年）"] == "2020年_不詳補完値"
pyramid_df = pop_value.loc[pyramid_cond, pyramid_cols]

In [ ]:
pop_pyramid = pyramid_df.sort_values("年齢（５歳階級）_時系列コード").assign(**
    {"人口": pyramid_df["値"].mask(pyramid_df["男女_時系列"] == "男", -pyramid_df["値"])}
).rename(columns={"年齢（５歳階級）_時系列": "年齢（５歳階級）", "男女_時系列": "性別"})
pop_pyramid.head()

In [ ]:
pyramid_fig = px.bar(
    pop_pyramid, 
    x="人口", 
    y="年齢（５歳階級）", 
    color="性別", 
    orientation="h"  # "h"は水平方向(横棒グラフ)
)
pyramid_fig.update_layout(width=900, height=600, font={"size": 14},
    xaxis=go.layout.XAxis(
       tickvals=[-4_000_000, -2_000_000, 0, 2_000_000, 4_000_000],
       ticktext=["400万","200万", "0", "200万", "400万"],
    ))
pyramid_fig.show()

### 人口推移

In [ ]:
pop_cols = ["時間軸（調査年）コード", "年齢（５歳階級）_時系列コード", "年齢（５歳階級）_時系列", "値"]
pop_cond = pop_value["表章項目"] == "人口"
pop_cond &= pop_value["男女_時系列"] == "総数"
pop_cond &= pop_value["年齢（５歳階級）_時系列"] != "総数"
pop_cond &= ~ pop_value["年齢（５歳階級）_時系列"].str.contains("再掲")
pop_cond &= ~ pop_value["時間軸（調査年）"].str.contains("不詳補完値")
pop_df = pop_value.loc[pop_cond, pop_cols]

In [ ]:
pop_df = pop_df.assign(**{
    "年": pop_df["時間軸（調査年）コード"].astype(int) // 1_000_000,
    "年齢（５歳階級）": pop_df["年齢（５歳階級）_時系列コード"] + "_" + pop_df["年齢（５歳階級）_時系列"]
})

In [ ]:
pop_colors = px.colors.sequential.dense[:3]
pop_colors += px.colors.sequential.matter[:10]
pop_colors += px.colors.sequential.deep[:11]
pop_fig = px.bar(pop_df, x="年", y="値", color="年齢（５歳階級）", color_discrete_sequence=pop_colors)
pop_fig.update_layout(width=900, height=700, font={"size": 14},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None}
)
pop_fig.show()

## 人口動態

- [人口動態調査](https://www.mhlw.go.jp/toukei/list/81-1.html)
- [公表](https://www.mhlw.go.jp/toukei/kouhyou/e-stat_81-1.xml)


In [ ]:
vital_statsDataId = "0003411561"
vital_meta = get_metainfo(appId, vital_statsDataId)
vital_metadata = vital_meta["GET_META_INFO"]["METADATA_INF"]
vital_total_num = vital_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
vital_total_num

In [ ]:
vital_data = get_statsdata(appId, vital_statsDataId)
vital_value_df = colname_to_japanese(cleansing_statsdata(vital_data))
vital_value_df.columns

### 出生数と死亡数


In [ ]:
vital_value = vital_value_df.assign(**{
    "年": vital_value["時間軸(年次)コード"].astype(int) // 1_000_000
})
vital_cond = vital_value["人口動態総覧"].isin(["出生数", "死亡数"])
vital_df = vital_value.loc[vital_cond, ["年", "人口動態総覧", "値"]]
vital_df.head()

In [ ]:
vital_delta = vital_value.loc[vital_value["人口動態総覧"] == "自然増減数", ["年", "値"]]
delta_pop = vital_delta.sort_values("年").rename(columns={"値": "自然増減数"})

In [ ]:
vital_fig = px.line(vital_df, x="年", y="値", color="人口動態総覧", markers=True)
vital_fig.add_bar(x=delta_pop["年"], y=delta_pop["自然増減数"], name="自然増減数")
vital_fig.update_layout(width=1200, height=650, font={"size": 24},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None}
)
vital_fig.show()

### 合計特殊出生率

-[人口置換水準](https://www.ipss.go.jp/syoushika/tohkei/Popular/Popular2020.asp?chap=0)


In [ ]:
tfr_cond = vital_value["人口動態総覧"] == "合計特殊出生率"
tfr_cond &= vital_value["値"].notna()
tfr_df = vital_value.loc[tfr_cond, ["年", "人口動態総覧", "値"]]
tfr_df.head()

In [ ]:
tfr_fig = px.line(tfr_df, x="年", y="値", color="人口動態総覧")
tfr_fig.add_trace(go.Scatter(
    x=[1947, 2021], 
    y=[2.07, 2.07],
    mode="lines", 
    line={"dash": "dash"},
    name="人口置換水準"))
tfr_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis= {"ticksuffix": "人", "title": None}
)
tfr_fig.show()

### 婚姻件数と離婚件数


In [ ]:
divorce_statsDataId = "0003411864"
divorce_meta = get_metainfo(appId, divorce_statsDataId)
divorce_metadata = divorce_meta["GET_META_INFO"]["METADATA_INF"]
divorce_total_num = divorce_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
divorce_total_num

In [ ]:
divorce_data = get_statsdata(appId, divorce_statsDataId)
divorce_value = colname_to_japanese(cleansing_statsdata(divorce_data))
divorce_value.columns

In [ ]:
divorce_cols = ["時間軸(年次)コード", "同居期間コード", "同居期間", "値"]
divorce_cond = divorce_value["単位"] == "件"
divorce_cond &= ~divorce_value["同居期間"].isin(["離婚件数_総数", "離婚件数_5年未満"])
divorce_df = divorce_value.loc[divorce_cond, divorce_cols]

In [ ]:
divorce_df = divorce_df.assign(**{
    "同居期間": divorce_df["同居期間コード"].str[-3:] + "_" + divorce_df["同居期間"],
    "年": divorce_df["時間軸(年次)コード"].astype(int) // 1_000_000
})

In [ ]:
marriage_cond = vital_value["人口動態総覧"].isin(["婚姻件数", "離婚件数"])
marriage_cond &= vital_value["年"] >= 1947
marriage_df = vital_value.loc[marriage_cond, ["年", "人口動態総覧", "値"]]
pivot_df = marriage_df.pivot(columns="人口動態総覧", index="年", values="値")
pivot_df.head()

In [ ]:
# サブプロットの作成
marriage_fig = make_subplots(rows=1, cols=2, subplot_titles=("婚姻・離婚件数", "離婚内訳"))
# 左側の婚姻・離婚件数グラフ
marriage_fig.add_trace(go.Scatter(
    x=pivot_df.index, 
    y=pivot_df["婚姻件数"],
    mode="lines", 
    name="婚姻件数"
), row=1, col=1)
marriage_fig.add_trace(go.Scatter(
    x=pivot_df.index, 
    y=pivot_df["離婚件数"],
    mode="lines", 
    name="離婚件数"
), row=1, col=1)
# 右側の離婚内訳エリアグラフ(matterのカラーが不足するので一つカラーコード追加)
divorce_colors = px.colors.sequential.matter + ["#EF553B"]
for n, t in enumerate(divorce_df["同居期間"].unique()):
    df_filtered = divorce_df[divorce_df["同居期間"] == t]
    marriage_fig.add_trace(go.Scatter(
        x=df_filtered["年"], 
        y=df_filtered["値"],
        name=t, 
        marker={"color": divorce_colors[n]},
        stackgroup="divorcegroup"
    ), row=1, col=2)
marriage_fig.update_layout(width=1200, height=550, font={"size": 14})
# 左側グラフのX軸とY軸の設定
marriage_fig.update_xaxes(ticksuffix="年", row=1, col=1)
marriage_fig.update_yaxes(tickformat=",", ticksuffix="人", row=1, col=1)
# 右側グラフのX軸とY軸の設定
marriage_fig.update_xaxes(ticksuffix="年", row=1, col=2)
marriage_fig.update_yaxes(tickformat=",", ticksuffix="人", row=1, col=2)
marriage_fig.show()

## 人口推計

- [人口推計](https://www.stat.go.jp/data/jinsui/index.html)
- [公表](https://www.stat.go.jp/data/kouhyou/e-stat_jinsui.xml)


In [ ]:
pop_estimates_statsDataId = "0003448228"
estimates_meta = get_metainfo(appId, pop_estimates_statsDataId)
estimates_metadata = estimates_meta["GET_META_INFO"]["METADATA_INF"]
estimates_total_num = estimates_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
estimates_total_num

In [ ]:
estimates_data = get_statsdata(appId, pop_estimates_statsDataId)
estimates_value = colname_to_japanese(cleansing_statsdata(estimates_data))
estimates_value.columns

In [ ]:
estimates_cols = ["男女別・性比", "年齢各歳コード", "年齢各歳", "値"]
estimates_cond = estimates_value["男女別・性比"].isin(["男", "女"])
estimates_cond &= estimates_value["人口"] == "総人口"
estimates_cond &= estimates_value["時間軸（年月日現在）"] == "2024年10月1日現在"
estimates_cond &= estimates_value["年齢各歳"] != "総数"
estimates_df = estimates_value.loc[estimates_cond, estimates_cols]

In [ ]:
estimates_df = estimates_df.assign(**{
    "人口": estimates_df["値"].mask(estimates_df["男女別・性比"] == "男", -estimates_df["値"]),
    "年齢": estimates_df["年齢各歳コード"].str[-3:].astype(int) - 1
})

In [ ]:
estimates_fig = px.bar(estimates_df, x="人口", y="年齢", color="男女別・性比", orientation="h")
estimates_fig.update_layout(width=1000, height=450, font={"size": 18},
    xaxis=go.layout.XAxis(
       tickvals=[-1000, -500, 0, 500, 1000],
       ticktext=["1000千","500千", "0", "500千", "1000千"],
    ),
    yaxis={"ticksuffix": "歳", "title": None}
)
estimates_fig.show()

## 将来推計人口

- [国立社会保障・人口問題研究](https://www.ipss.go.jp/)
- [国立社会保障・人口問題研究所のサイトの「将来推計人口・世帯数」のページ](https://www.ipss.go.jp/syoushika/tohkei/Mainmenu.asp)
- [著作権・リンク許可について](https://www.ipss.go.jp/site-ad/link/anlink.html)
- [openpyxl](https://openpyxl.readthedocs.io/en/stable/)


In [ ]:
#!pip install openpyxl

In [ ]:
pop_projections_df = pd.read_excel(data_path / "1-1.xlsx", header=[0, 1], skiprows=2, skipfooter=2)

In [ ]:
url_path = "https://www.ipss.go.jp/pp-zenkoku/j/zenkoku2023/db_zenkoku2023/s_tables/1-1.xlsx"
#pop_projections_df = pd.read_excel(url_path, header=[0, 1], skiprows=2, skipfooter=2)
pop_projections_df.head()

In [ ]:
pop_projections_df = pop_projections_df.set_axis(
    [(i + j.replace("Unnamed: 0_level_1", "和暦").replace("Unnamed: 1_level_1", "西暦"))
      .replace("　", "")
      for i, j in pop_projections_df.columns],
    axis=1
)

In [ ]:
value_vars = ["人口（1,000人）0～14歳", "人口（1,000人）15～64歳", "人口（1,000人）65歳以上"]
pop_tidy_df = pd.melt(
    pop_projections_df, 
    id_vars="年次西暦", 
    value_vars=value_vars, 
    var_name="年齢", 
    value_name="人口"
)
pop_tidy_df.tail()

In [ ]:
pop_totals = pop_tidy_df[["年次西暦", "人口"]].groupby("年次西暦").sum().reset_index()
totals_per_10year = pop_totals.loc[pop_totals["年次西暦"] % 10 == 0]
annotations = (totals_per_10year["人口"] // 1000).map(lambda x: f"{x:.0f}百万")

In [ ]:
pop_projections_fig = px.bar(pop_tidy_df, x="年次西暦", y="人口", color="年齢")
pop_projections_fig.add_trace(go.Scatter(
    x=totals_per_10year["年次西暦"],
    y=totals_per_10year["人口"] + 5000,
    text=annotations,
    mode="text",
    showlegend=False
))
pop_projections_fig.update_layout(width=1000, height=400, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "千人"},
)
pop_projections_fig.show()

## 世界の人口

- [World Population Prospects](https://population.un.org/wpp/)
- [World Population Prospects サイト 人口のグラフ](https://population.un.org/wpp/graphs)
- [download](https://population.un.org/wpp/downloads?folder=Standard%20Projections&group=CSV%20format)
- [国際標準化機構](https://www.iso.org/iso-3166-country-codes.html)


In [ ]:
world_pop = pd.read_csv(data_path / "WPP2024_TotalPopulationBySex.csv", low_memory=False)
world_pop.columns

In [ ]:
world_pop["ISO3_code"].unique()[-10:]

In [ ]:
world_pop_cond = world_pop["ISO3_code"].notna()
world_pop_cond &= world_pop["Variant"] == "Medium"
pop_med = world_pop[world_pop_cond]
pop_med.iloc[:5, np.r_[0:5, len(pop_med.columns)-5:len(pop_med.columns)]]

In [ ]:
pop_med["Location"].nunique()

### 将来人口推移


In [ ]:
pop2020 =  pop_med.loc[pop_med["Time"]==2020, ["ISO3_code", "Location", "PopTotal"]]
pop2020 = pop2020.assign(**{"Rank2020": pop2020["PopTotal"].rank(ascending=False)})
pop_rank = pop2020.nlargest(20, "PopTotal")

In [ ]:
sorted_df = pop_rank.sort_values("PopTotal")
pop_bar_fig = go.Figure(go.Bar(
    x=sorted_df["PopTotal"], 
    y=sorted_df["Location"],
    text=sorted_df["PopTotal"].astype(int), 
    textposition="outside",
    orientation="h",
))
max_value = max(sorted_df["PopTotal"]) * 1.2  # 最大値の120%までx軸を拡張する
pop_bar_fig.update_layout(width=800, height=600, font={"size": 14},
    xaxis={
        "tickformat": ",.0f", 
        "ticksuffix": "千人",
        "title": "人口", 
        "range": [0, max_value]  # x軸の範囲を手動で設定
    },
    yaxis_title="国",
    margin={"l": 100, "r": 50}
)
pop_bar_fig.show()

In [ ]:
pop_top20 = pop_med.merge(pop_rank[["Location", "Rank2020"]], on="Location", how="inner")
pop_top20.sort_values(["Rank2020", "Time"], inplace=True)
pop_colors = plotly.colors.qualitative.Plotly + plotly.colors.qualitative.Set3
pop_line_fig = px.line(
    pop_top20, 
    x="Time", 
    y="PopTotal",
    color="Location", 
    color_discrete_sequence=pop_colors
)
pop_line_fig.update_layout(width=900, height=700, font={"size": 16},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "千人", "title": None}
)
pop_line_fig.show()

### 人口の世界地図

In [ ]:
choropleth_fig = px.choropleth(
    pop2020, 
    color="PopTotal",
    locations="ISO3_code", # 位置を ISO 3166-1 alpha-3 形式で指定
    hover_name="Location")  # ホバーツールのタイトル
choropleth_fig.show()

In [ ]:
pop_geo_fig = px.scatter_geo(
    pop_med, 
    size="PopTotal",  # 要素のサイズ
    locations="ISO3_code",  # 位置を ISO 3166-1 alpha-3 形式で指定
    hover_name="Location",  # ホバーツールのタイトル
    animation_frame="Time")  # アニメーション
pop_geo_fig.show()

## 生命表と寿命

- [生命表](https://www.mhlw.go.jp/toukei/list/list54-57.html)
- [公表](https://www.mhlw.go.jp/toukei/kouhyou/e-stat_55-18.xml)
- [標準生命表](https://www.actuaries.jp/lib/standard-life-table/)

### 完全生命表と簡易生命表


In [ ]:
life_table_statsDataId = "0003109570" 
life_meta = get_metainfo(appId, life_table_statsDataId)
life_metadata = life_meta["GET_META_INFO"]["METADATA_INF"]
life_total_num = life_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
life_total_num

In [ ]:
life_data = get_statsdata(appId, life_table_statsDataId)
life_value = colname_to_japanese(cleansing_statsdata(life_data))
life_value.columns

In [ ]:
life_cols = ["表章項目", "性別", "年齢区分（完全生命表）", "値"]
life_df = life_value.loc[life_value["時間軸（年次）"]=="2015年", life_cols]

In [ ]:
life_df = life_df.assign(**{
    "年齢": life_df["年齢区分（完全生命表）"].str[:-1].astype(int),
    "週月年": life_df["年齢区分（完全生命表）"].str[-1]
})
life_df = life_df.loc[life_df["週月年"]=="年", ["表章項目", "性別", "年齢", "値"]]

In [ ]:
pivot_df = life_df.pivot(columns="表章項目", index=["性別", "年齢"], values="値")
pivot_df = (pivot_df
    .set_axis([c.replace("\u3000", "") for c in pivot_df.columns], axis=1)
    .reset_index()
)
pivot_df.head()

In [ ]:
def calc_leb(df, gender):
    return df.loc[
        (df["年齢"]==0)&(df["性別"]==gender), "平均余命(ex)"
    ].iat[0]
lifespan_male = calc_leb(pivot_df, "男")
lifespan_female = calc_leb(pivot_df, "女")
print("男:", lifespan_male, ", 女:", lifespan_female)

In [ ]:
def calc_med_lifespan(df, gender):
    data = df[df["性別"] == gender].sort_values("年齢")
    half = data["生存数(lx)"].iloc[0] / 2
    # 厳密な計算ではありません
    return data[data["生存数(lx)"] < half]["年齢"].iloc[0]
print("男:", calc_med_lifespan(pivot_df, "男"), ", 女:", calc_med_lifespan(pivot_df, "女"))

In [ ]:
ndx = pivot_df[["性別", "年齢", "死亡数(ndx)"]]
male_ndx = ndx.loc[ndx["性別"]=="男"]
female_ndx = ndx.loc[ndx["性別"]=="女"]
life_fig = go.Figure(data=[
    go.Bar(x=male_ndx["年齢"], y=male_ndx["死亡数(ndx)"], name="男性"),
    go.Bar(x=female_ndx["年齢"], y=female_ndx["死亡数(ndx)"], name="女性")
])
life_fig.add_trace(go.Scatter(
    x=[lifespan_male, lifespan_male], 
    y=[0, 5000],
    name="男性平均寿命", 
    mode="lines", line={"dash": "dash"},
    marker_color=px.colors.qualitative.Plotly[0]))
life_fig.add_trace(go.Scatter(
    x=[lifespan_female, lifespan_female], 
    y=[0, 5000],
    name="女性平均寿命", 
    mode="lines", line={"dash": "dash"},
    marker_color=px.colors.qualitative.Plotly[1]))
life_fig.update_layout(width=800, height=450, font={"size": 18},
    xaxis={"ticksuffix": "歳", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None},
    title="10万人あたり死亡数", 
    barmode="group"
)
life_fig.show()

### 世界の平均寿命・健康寿命

- [GHO](https://www.who.int/data/gho)
- [OData API](https://www.who.int/data/gho/info/gho-odata-api)
- [Terms and conditions for WHO datasets](https://data.who.int/about/data/terms-and-conditions)


In [ ]:
base_url = "https://ghoapi.azureedge.net/api/"
contain_name = "Life expectancy"
contain_params = {"$filter": f"contains(IndicatorName,'{contain_name}')"}
indicator_contain_res = requests.get(base_url + "Indicator", params=contain_params)
indicator_contain_res.json()

In [ ]:
eq_name = "Life expectancy at birth (years)"
eq_params = {"$filter": f"IndicatorName eq '{eq_name}'"}
indicator_eq_res = requests.get(base_url + "Indicator", params=eq_params)
indicator_eq_res.json()

In [ ]:
lifeexp_res = requests.get(base_url + "WHOSIS_000001")
lifeexp_value = pd.json_normalize(lifeexp_res.json(), record_path=["value"])
lifeexp_value.columns

In [ ]:
lifeexp_cond = lifeexp_value["TimeDim"] == 2020
lifeexp_cond &= lifeexp_value["Dim1"] == "SEX_BTSX"
lifeexp_df = lifeexp_value.loc[lifeexp_cond, ["Dim1", "SpatialDim", "NumericValue"]]
lifeexp_df.head()

In [ ]:
country_df = pop2020.merge(lifeexp_df, left_on="ISO3_code", right_on="SpatialDim", how="inner")
top30_df = country_df.nlargest(30, "PopTotal").loc[:, ["Location", "PopTotal", "NumericValue"]]

In [ ]:
lifeexp_fig = px.scatter(
    x=top30_df["PopTotal"], 
    y=top30_df["NumericValue"], 
    text=top30_df["Location"]
)
lifeexp_fig.update_traces(textposition="bottom right")
lifeexp_fig.update_layout(width=800, height=600, font={"size": 14},
    xaxis={
        "tickformat": ",.0f", 
        "ticksuffix": "千人", 
        "title": "人口", 
        "range": [0, top30_df["PopTotal"].max() * 1.1]
    },
    yaxis={"ticksuffix": "歳", "title": "平均寿命"}
)
lifeexp_fig.show()

In [ ]:
contain_healthy = "Healthy"
healthy_params = {"$filter": f"contains(IndicatorName,'{contain_name}')"}
healthy_res = requests.get(base_url + "Indicator", params=contain_params)
healthy_res.json()

In [ ]:
hale_res = requests.get(base_url + "WHOSIS_000002")
hale_value = pd.json_normalize(hale_res.json(), record_path=["value"])
cond = hale_value["TimeDim"] == 2020
cond &= hale_value["Dim1"] != "BTSX"
hale_df = hale_value.loc[cond, ["Dim1", "SpatialDim", "NumericValue"]]
hale_df.head()


- [保健指標評価研究所]((https://www.healthdata.org/))
- [ihme_data](https://ghdx.healthdata.org/ihme_data)
- [terms-and-conditions](https://www.healthdata.org/data-tools-practices/data-practices/terms-and-conditions)


## 疾病と感染症

### 人口動態と死因の国際疾病分類

- [International
Statistical Classification of Diseases and Related Health Problems：ICD](https://www.mhlw.go.jp/toukei/sippei/)

#### 死因年次推移


In [ ]:
disease_statsDataId = "0003411656"
disease_meta = get_metainfo(appId, disease_statsDataId)
disease_metadata = disease_meta["GET_META_INFO"]["METADATA_INF"]
disease_total_num = disease_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
disease_total_num

In [ ]:
disease_data = get_statsdata(appId, disease_statsDataId)
disease_value = colname_to_japanese(cleansing_statsdata(disease_data))
disease_value.columns

In [ ]:
disease_cond = disease_value["死因年次推移分類"] != "総数"
disease_cond &= disease_value["単位"] == "人口10万対"
disease_cond &= disease_value["性別"] == "総数"
disease_df = disease_value.loc[
    disease_cond, ["死因年次推移分類", "時間軸(年次)コード", "値"]
].sort_values(["時間軸(年次)コード", "死因年次推移分類"])
disease_df = disease_df.assign(**{
    "年": disease_df["時間軸(年次)コード"].astype(int) // 1_000_000
})
disease_df.head()

In [ ]:
disease_colors = px.colors.qualitative.Plotly + px.colors.qualitative.Set3
disease_fig = px.line(disease_df, x="年", y="値",
    color="死因年次推移分類", color_discrete_sequence=disease_colors)
disease_fig.update_layout(width=900, height=600, font={"size": 16},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"ticksuffix": "人", "tickformat": ",.0f", "title": "人口10万対"}
)
disease_fig.show()

#### 年齢ごと死因


In [ ]:
icd_statsDataId = "0003411691"
icd_meta = get_metainfo(appId, icd_statsDataId)
icd_metadata = icd_meta["GET_META_INFO"]["METADATA_INF"]
icd_total_num = icd_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
icd_total_num

In [ ]:
%%time
icd_statsDataIds = ["0003411691", "0003411692", "0003411693", "0004018620"]
icd_params = {"lvCat03": "3", "cdTime": "2023000000"}
cat = "死因基本分類"
icd_dfs = []
for n, sdi in enumerate(icd_statsDataIds):
    print(sdi, "取得開始")
    icd_data = get_statsdata(appId, sdi, params=icd_params)
    icd_value = colname_to_japanese(cleansing_statsdata(icd_data))
    icd_cols = ["年齢(5歳階級)", "性別", f"死因基本分類_{n+1}", "値"]
    icd_cond = icd_value["性別"] != "総数"
    icd_cond &= ~icd_value["年齢(5歳階級)"].isin(["総数", "不詳"])
    icd_df = icd_value.loc[icd_cond, icd_cols].rename(columns={f"{cat}_{n+1}": cat})
    icd_dfs += [icd_df]
icd = pd.concat(icd_dfs)
icd.head()

In [ ]:
age_dict = {
    "0～4歳": ["0～4歳"],
    "5～9歳": ["5～9歳"],
    "10～19歳": ["10～14歳", "15～19歳"],
    "20～29歳": ["20～24歳", "25～29歳"],
    "30～39歳": ["30～34歳", "35～39歳"],
    "40～49歳": ["40～44歳", "45～49歳"],
    "50～69歳": ["50～54歳", "55～59歳", "60～64歳", "65～69歳"],
    "70～89歳": ["70～74歳", "75～79歳", "80～84歳", "85～89歳"],
    "90～歳": ["90～94歳", "95～99歳", "100歳以上"],
}

In [ ]:
def top_causes_by_age(icd, age_dict, cat, sex, top_n=5):
    death_cause = []
    for age_label, age_list in age_dict.items():
        cond = icd["年齢(5歳階級)"].isin(age_list) & (icd["性別"] == sex)
        icd_rank = (icd
            .loc[cond, [cat, "値"]]
            .groupby(cat, as_index=False).sum()
            .sort_values("値", ascending=False)
        )
        top = icd_rank[cat].head(top_n).reset_index(drop=True)
        top.name = age_label
        death_cause.append(top)
    return pd.concat(death_cause, axis=1)
top_causes_by_age(icd, age_dict, cat, sex="男")

In [ ]:
top_causes_by_age(icd, age_dict, cat, sex="女")

### COVID-19


#### 厚生労働省のCOVID-19データ

- [厚生労働省のCOVID-19 のオープンデータのサイト](https://www.mhlw.go.jp/stf/covid-19/open-data.html)


In [ ]:
covid_path = data_path / "severe_cases_daily.csv"
#covid_path = "https://covid19.mhlw.go.jp/public/opendata/severe_cases_daily.csv"
covid_df = pd.read_csv(covid_path, parse_dates=["Date"])
covid_tidy = (covid_df[["Date", "ALL"]]
    .rename(columns={"ALL": "重症者数の推移"})
    .set_index("Date")
    .resample("MS").sum()
    .reset_index()
)
covid_tidy.head()

#### WHOのCOVID-19データ

- [WHO のサイト](https://data.who.int/dashboards/covid19/data)


In [ ]:
csvname = "WHO-COVID-19-global-daily-data.csv"
who_covid_data = pd.read_csv(data_path / csvname, parse_dates=["Date_reported"])
who_covid_data.head()

In [ ]:
max_deaths = (who_covid_data
    .sort_values("Date_reported")[["Country", "Cumulative_deaths"]]
    .groupby("Country").last()
    .reset_index()
)
rank_deaths = max_deaths.nlargest(25, "Cumulative_deaths")
who_covid_daily = rank_deaths.merge(who_covid_data, on="Country", suffixes=("_rank", ""))
who_covid_monthly = (who_covid_daily[["Date_reported", "Country", "New_deaths"]]
    .groupby([pd.Grouper(key="Date_reported", freq="ME"), "Country"]).sum()
    .reset_index()
)

In [ ]:
covid_fig = px.line(who_covid_monthly, x="Date_reported", y="New_deaths", 
    color="Country", color_discrete_sequence=px.colors.qualitative.Alphabet)
covid_fig.update_layout(width=1200, height=800, font={"size": 16},
    xaxis={"ticksuffix": "年"},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人"}
)
covid_fig.show()